In [31]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [148]:
data = pd.read_csv('movie_bd_v5.csv')

data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1731,tt0408839,60000000,127766650,The Heartbreak Kid,Ben Stiller|Malin Ã…kerman|Michelle Monaghan|J...,Bobby Farrelly|Peter Farrelly,He waited all his life to get married. Too bad...,"Eddie, the 40-year-old confirmed bachelor fina...",116,Comedy,Conundrum Entertainment|Davis Entertainment|Dr...,10/5/2007,5.6,2007
113,tt3164256,15000000,3020664,Rock the Kasbah,Bill Murray|Zooey Deschanel|Bruce Willis|Kate ...,Barry Levinson,Opportunity rocks where you least expect it.,A washed-up music producer finds one last shot...,100,Comedy|Music|War,Shangri-La Entertainment|QED International|Dun...,10/22/2015,5.1,2015
791,tt1265990,16000000,40492759,The Roommate,Minka Kelly|Leighton Meester|Cam Gigandet|Aly ...,Christian E. Christiansen,"2,000 colleges. 8 million roommates. Which one...","When Sara (Minka Kelly), a young design studen...",91,Thriller|Drama|Horror,Vertigo Entertainment|Screen Gems,2/4/2011,5.5,2011
811,tt1650043,21000000,72417394,Diary of a Wimpy Kid: Rodrick Rules,Zachary Gordon|Devon Bostick|Rachael Harris|Ro...,David Bowers,Welcome to the next grade,"Back in middle school after summer vacation, G...",99,Family|Comedy,Fox 2000 Pictures|Color Force,3/25/2011,6.4,2011
1344,tt0356910,110000000,478207520,Mr. & Mrs. Smith,Angelina Jolie|Brad Pitt|Vince Vaughn|Adam Bro...,Doug Liman,Smart and sexy.,After five (or six) years of vanilla-wedded bl...,120,Action|Comedy|Drama|Thriller,20th Century Fox|Weed Road Pictures|Dutch Oven...,6/7/2005,6.5,2005


In [33]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [34]:
# Предобработка
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


In [151]:
answers = {} # создадим словарь для ответов


# тут другие ваши предобработки колонок например:
def split_str(cast : str) -> str:
    return [str(actor) for actor in cast.split('|')]
def is_profit(row):
    if row['budget'] < row.revenue:
        return True
    else:
        return False

def count_genres(data : pd.DataFrame) -> [set,list]:
  c = Counter()
  for row in data.list_genre:
    for genre in row:
      c[genre] += 1
  return set(c.elements()),c



data['list_cast'] = data.cast.apply(split_str)
#data.production_companies = data.production_companies.apply(split_str)

#data.genres = data.genres.apply(split_str)

# посчитать пропущенные значения
data.isna().sum()

#Привести к типу Datetime
data.release_date = pd.to_datetime(data.release_date)
# Добавить колонку окупаемости
data['is_profit'] = data.apply(is_profit, axis = 1)
data['profit'] = data.apply(lambda x: x['revenue'] - x['budget'], axis = 1)
data['list_director'] = data.director.str.split('|')
data['list_genre'] = data.genres.str.split('|')
data['list_company'] = data.production_companies.str.split('|')

data.info()
#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [36]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа +
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)' 
# если ответили верно, можете добавить комментарий со значком "+"

In [37]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()][['imdb_id','budget','original_title']]

,imdb_id,budget,original_title
723,tt1298650,380000000,Pirates of the Caribbean: On Stranger Tides


ВАРИАНТ 2

In [38]:
# можно добавлять разные варианты решения
data.sort_values(by=['budget'], ascending=False).head(1)[['imdb_id','budget','original_title','runtime']]

,imdb_id,budget,original_title,runtime
723,tt1298650,380000000,Pirates of the Caribbean: On Stranger Tides,136


# 2. Какой из фильмов самый длительный (в минутах)?

In [39]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = 'Gods and Generals (tt0279111)'

In [40]:
data[data.runtime == data.runtime.max()][['imdb_id','budget','original_title','runtime']]

,imdb_id,budget,original_title,runtime
1157,tt0279111,56000000,Gods and Generals,214


In [41]:
# вариант второй
data.sort_values(by=['runtime'], ascending=False).head(1)[['imdb_id','budget','original_title','runtime']]


,imdb_id,budget,original_title,runtime
1157,tt0279111,56000000,Gods and Generals,214


# 3. Какой из фильмов самый короткий (в минутах)?





In [42]:
answers['3'] = 'Winnie the Pooh  (tt1449283)'

In [43]:
data[data.runtime == data.runtime.min()][['imdb_id','budget','original_title','runtime']]

,imdb_id,budget,original_title,runtime
768,tt1449283,30000000,Winnie the Pooh,63


In [44]:
# Второй вариант 
data.sort_values(by=['runtime']).head(1)[['imdb_id','budget','original_title','runtime']]

,imdb_id,budget,original_title,runtime
768,tt1449283,30000000,Winnie the Pooh,63


In [45]:
# 4. Какова средняя длительность фильмов?
answers['4'] = '110'

In [46]:
round(data.runtime.mean())

110.0

# 5. Каково медианное значение длительности фильмов? 

In [47]:
answers['5'] = '107'

In [48]:
round(data.runtime.median())

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [49]:
answers['6'] = 'Avatar (tt0499549)'

In [50]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data[(data.is_profit == True) & (data.profit == max(data.profit))][['imdb_id','budget','revenue','profit','original_title','runtime']]

,imdb_id,budget,revenue,profit,original_title,runtime
239,tt0499549,237000000,2781505847,2544505847,Avatar,162


# 7. Какой фильм самый убыточный? 

In [51]:
answers['7'] = 'The Lone Ranger (tt1210819)'

In [52]:
data[(data.is_profit == False) & (data.profit == min(data.profit))][['imdb_id','budget','revenue','profit','original_title','runtime']]

,imdb_id,budget,revenue,profit,original_title,runtime
1245,tt1210819,255000000,89289910,-165710090,The Lone Ranger,149


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [53]:
answers['8'] = '1478'

In [54]:
data[data.is_profit == True].is_profit.count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [55]:
answers['9'] = 'The Dark Knight (tt0468569)'

In [56]:
max_revenue = data.groupby(['release_year'])['revenue'].max()
data[data.revenue == max_revenue.loc[2008]]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,is_profit,profit
599,tt0468569,185000000,1001921825,The Dark Knight,"[Christian Bale, Michael Caine, Heath Ledger, ...",Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,"[Drama, Action, Crime, Thriller]","[DC Comics, Legendary Pictures, Warner Bros., ...",2008-07-16,8.1,2008,True,816921825


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [57]:
answers['10'] = 'The Lone Ranger (tt1210819)'

In [58]:
mins_profit = data.groupby(['release_year'])[['profit']].min()
mins_our_years = mins_profit.loc[[2012,2013,2014]].min()
data[data.profit == mins_our_years[0]]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,is_profit,profit
1245,tt1210819,255000000,89289910,The Lone Ranger,"[Johnny Depp, Armie Hammer, William Fichtner, ...",Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,"[Action, Adventure, Western]","[Walt Disney Pictures, Jerry Bruckheimer Films...",2013-07-03,6.0,2013,False,-165710090


# 11. Какого жанра фильмов больше всего?

In [196]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
# либо функцией counter посчитать
unique_genres = count_genres(data)
print(unique_genres[0])

Counter({'Drama': 782, 'Comedy': 683, 'Thriller': 596, 'Action': 582, 'Adventure': 415, 'Crime': 315, 'Romance': 308, 'Family': 260, 'Science Fiction': 248, 'Fantasy': 222, 'Horror': 176, 'Mystery': 168, 'Animation': 139, 'Music': 64, 'History': 62, 'War': 58, 'Western': 19, 'Documentary': 8, 'Foreign': 2})


ВАРИАНТ 2

In [206]:
genres_exploded = data.explode('list_genre')
genres_exploded.list_genre.value_counts(sort=True).head(1)

Drama    782
Name: list_genre, dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [200]:
profit_films = data[data.profit > 0]
profit_films_exploded = profit_films.explode('list_genre')
profit_films_exploded.list_genre.value_counts().head(1)

Drama    560
Name: list_genre, dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [63]:
data.groupby(['director'])['revenue'].sum().sort_values(ascending=False).head(1)

director
Peter Jackson    6490593685
Name: revenue, dtype: int64


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [129]:
data_action = data[data.genres.str.contains('Action')][['list_director']]
data_action_exploded = data_action.explode('list_director')
data_action_exploded.list_director.value_counts().head(1)

Robert Rodriguez    9
Name: list_director, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [207]:
data_2012 = data[(data.release_year == 2012) & (data.profit > 0)]
data_exploded = data_2012.explode('list_cast')
data_exploded.groupby(by = 'list_cast')['revenue'].agg('sum').sort_values(ascending=False).head(1)

list_cast
Chris Hemsworth    1982643990
Name: revenue, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [186]:
high_budget = data[data.budget > data.budget.mean()]
high_budget_exploded = high_budget.explode('list_cast')
high_budget_exploded.list_cast.value_counts().head(1)

Matt Damon    18
Name: list_cast, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [208]:
cage_genres = data[data.cast.str.contains('Nicolas Cage')][['list_genre']]
cage_genres_explode= cage_genres.explode('list_genre')
cage_genres_explode.list_genre.value_counts().head(1)

Action    17
Name: list_genre, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [165]:
data[data.production_companies.str.contains('Paramount Pictures')][['imdb_id','original_title','profit']].sort_values('profit').head(1)

,imdb_id,original_title,profit
925,tt0267626,K-19: The Widowmaker,-64831034


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

# 20. Какой самый прибыльный год для студии Warner Bros?

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

# 23. Для какого режиссера зима – самое продуктивное время года? 

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

# Submission

In [60]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh  (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)'}

In [61]:
# и убедиться что ни чего не пропустил)
len(answers)

10